### Flatten the Data from the JSON Files

In [ ]:
import os
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Where files live
os.chdir(r'/content/drive/MyDrive/Graduate School/IST718 Big Data Analysis/Final Project/Unzipped Files/Starter File/Subset')
path ='/content/drive/MyDrive/Graduate School/IST718 Big Data Analysis/Final Project/Unzipped Files/Starter File/Subset'
savedir = '/content/drive/MyDrive/Graduate School/IST718 Big Data Analysis/Final Project/flattenedOutput'


In [ ]:
#The loop to end all loops
path = os.getcwd()
for subdir, dirs, files in os.walk(path):
  for f in files:
    medicines =[] #list to collect medicines in the report
    reactions =[] #list to collect recations in the report
    if f.endswith('.json'):
      with open(os.path.join(subdir,f)) as data_file:
        d = json.load(data_file)
        df = pd.json_normalize(d['results'], max_level=None)
        del data_file
        del d
        #Flatten Additional Columns
        df['patient.reaction'] = df['patient.reaction'].apply(lambda x: pd.json_normalize(x))
        df['patient.drug'] = df['patient.drug'].apply(lambda x: pd.json_normalize(x))
        df = df.drop(columns=['reporttype', 'safetyreportversion', 'receivedateformat', 'transmissiondateformat',
                  'fulfillexpeditecriteria','receiptdateformat', 'duplicate', 'receivedate', 'transmissiondate',
                  'primarysource.reportercountry', 'reportduplicate.duplicatenumb', 'receiver.receivertype',
                  'receiver.receiverorganization', 'sender.sendertype', 'sender.senderorganization',
                  'patient.summary.narrativeincludeclinical', 'primarysource.literaturereference'])
        df = df[(df['primarysource.qualification']=='3')|(df['primarysource.qualification']=='2')|(df['primarysource.qualification']=='1')].reset_index()

#Collect the Medicines
        for i in range(0, len(df['safetyreportid'])):
          report = pd.DataFrame(df['patient.drug'][i])
          for l in range(0, len(report['medicinalproduct'])):
            d = {'safetyreportid':df['safetyreportid'][i], 'medicinalproduct':report['medicinalproduct'][l]}
            medicines.append(d)

#Collect the Reactions
        for i in range(0, len(df['safetyreportid'])):
          report = pd.DataFrame(df['patient.reaction'][i])
          for l in range(0, len(report['reactionmeddrapt'])):
            d={'safetyreportid':df['safetyreportid'][i], 'reactionmedrapt':report['reactionmeddrapt'][l]}
            reactions.append(d)
        df = df.drop(columns=['patient.drug','patient.reaction'])
        del d
        print(f)
        m=pd.DataFrame(medicines)
        r=pd.DataFrame(reactions)
        df=df.merge(r, how = 'left', on = 'safetyreportid')
        df=df.merge(m, how = 'left', on = 'safetyreportid')
        df.to_csv(os.path.join(savedir,f[:-5]+'.csv'))
        del df
        del r
        del m